In [1]:
%pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00


In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [3]:
#Preparing google drive ot import the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/images.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [5]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [6]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [7]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

surprise completed
angry completed
neutral completed
happy completed
disgust completed
fear completed
sad completed


In [8]:
print(train)

                                 image     label
0       images/train/surprise/3283.jpg  surprise
1      images/train/surprise/16185.jpg  surprise
2      images/train/surprise/15314.jpg  surprise
3       images/train/surprise/5722.jpg  surprise
4      images/train/surprise/13393.jpg  surprise
...                                ...       ...
28816        images/train/sad/6662.jpg       sad
28817       images/train/sad/16173.jpg       sad
28818       images/train/sad/33195.jpg       sad
28819       images/train/sad/31407.jpg       sad
28820       images/train/sad/31894.jpg       sad

[28821 rows x 2 columns]


In [9]:
test = pd.DataFrame()
test['images'], test['labels'] = createdataframe(TEST_DIR)

surprise completed
angry completed
neutral completed
happy completed
disgust completed
fear completed
sad completed


In [10]:
print(test)
print(test['images'])

                              images    labels
0     images/test/surprise/34134.jpg  surprise
1     images/test/surprise/35237.jpg  surprise
2     images/test/surprise/13876.jpg  surprise
3     images/test/surprise/10246.jpg  surprise
4     images/test/surprise/26483.jpg  surprise
...                              ...       ...
7061       images/test/sad/27117.jpg       sad
7062        images/test/sad/4594.jpg       sad
7063         images/test/sad/359.jpg       sad
7064       images/test/sad/20192.jpg       sad
7065       images/test/sad/10769.jpg       sad

[7066 rows x 2 columns]
0       images/test/surprise/34134.jpg
1       images/test/surprise/35237.jpg
2       images/test/surprise/13876.jpg
3       images/test/surprise/10246.jpg
4       images/test/surprise/26483.jpg
                     ...              
7061         images/test/sad/27117.jpg
7062          images/test/sad/4594.jpg
7063           images/test/sad/359.jpg
7064         images/test/sad/20192.jpg
7065         images/t

In [11]:
from tqdm.notebook import tqdm

In [12]:
# pre-processing of images  loading grayscale images, normalizing pixel, and reshaping them to the required format (48x48x1)
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


In [15]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [18]:
test_features = extract_features(test['images'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [19]:
x_train = train_features/255.0
x_test = test_features/255.0

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [22]:
y_train = le.transform(train['label'])
y_test = le.transform(test['labels'])

In [23]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [24]:
#Build CNN model
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
#compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy' ])

In [22]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 250, validation_data = (x_test,y_test))

Epoch 1/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 34s 98ms/step - accuracy: 0.2358 - loss: 1.8403 - val_accuracy: 0.2583 - val_loss: 1.8110
Epoch 2/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.2510 - loss: 1.8073 - val_accuracy: 0.2581 - val_loss: 1.7645
Epoch 3/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.2692 - loss: 1.7517 - val_accuracy: 0.3306 - val_loss: 1.6487
Epoch 4/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.3356 - loss: 1.6533 - val_accuracy: 0.4247 - val_loss: 1.4975
Epoch 5/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.4015 - loss: 1.5326 - val_accuracy: 0.4645 - val_loss: 1.4139
Epoch 6/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.4403 - loss: 1.4569 - val_accuracy: 0.4983 - val_loss: 1.3347
Epoch 7/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 48ms/step - accuracy: 0.4589 - loss: 1.4102 - val_accuracy: 0.4990 - val_loss: 1.3010
Epoch 8/250
226/226 ━━━━━━━━━━━━━━━━━━━━ 20s 46ms/step - accuracy: 0.4738 - loss: 1

In [23]:
model_json = model.to_json()
with open("detection.json",'w') as json_file:
    json_file.write(model_json)
model.save("detection.h5")

In [24]:
from keras.models import model_from_json
json_file = open("detection.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("detection.h5")
label = ['angry','disgust','fear','happy','neutral','sad','surprise']
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [30]:
#predicting the model by selecting an image from train data
image = '/content/images/test/happy/10019.jpg'
print("emotion is happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
image = '/content/images/test/angry/10079.jpg'
print("emotion is angry")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

emotion is happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
model prediction is  happy
emotion is angry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
model prediction is  angry
